In [1]:
import datetime
import time
import functools

import pandas as pd
import numpy as np

import nba_py
import nba_py.game
import nba_py.player
import nba_py.team

import pymysql
from sqlalchemy import create_engine

In [2]:
conn = create_engine('mysql+pymysql://root:@localhost:3306/nba_stats')

try:
    # read sql table of game header
    game_header = pd.read_sql_table('game_header', conn)
    length_1 = len(game_header)
    print(str(length_1) + ' games loaded.')
    # set begin date to the newest date in sql table
    begin = datetime.datetime.strptime(game_header.iloc[-1]['GAME_DATE_EST'][:10], "%Y-%m-%d").date()
except ValueError:
    print('no table yet!')
    length_1 = 0
    # if no table yet, set begin date to 2012-10-29
    begin = datetime.date(2012, 10, 29)
    # grab game headers of begining date
    game_header = nba_py.Scoreboard(month = begin.month, 
                                        day = begin.day, 
                                        year = begin.year, league_id = '00', offset = 0).game_header()

# set end date to today
end = datetime.date.today()

for i in range((end - begin).days + 1):
    # grab game headers from begin date to end date
    day = begin + datetime.timedelta(days = i)
    game_header = game_header.append(nba_py.Scoreboard(month = day.month, 
                                                       day = day.day, 
                                                       year = day.year, 
                                                       league_id = '00', 
                                                       offset = 0).game_header())
    print(str(day) + ' finished!    ' + str(datetime.datetime.now().time())[:8])

length_2 = len(game_header)
# drop the duplicate by game id
game_header = game_header.drop_duplicates('GAME_ID')
length_3 = len(game_header)
print(str(length_2 - length_3) + ' duplicates droped.')
print(str(length_3 - length_1) + ' games added.')

# sort game headers by game id ascending
game_header = game_header.sort_values('GAME_ID')

# commit new game headers to sql table
game_header.to_sql('game_header', conn, index = False, if_exists = 'replace')
print(str(length_3) + ' game headers commit complete!')

7019 games loaded.
2017-06-12 finished!    09:55:45
2017-06-13 finished!    09:55:47
2017-06-14 finished!    09:55:48
2017-06-15 finished!    09:55:50
2017-06-16 finished!    09:55:52
2017-06-17 finished!    09:55:54
2017-06-18 finished!    09:55:57
2017-06-19 finished!    09:55:58
2017-06-20 finished!    09:56:00
2017-06-21 finished!    09:56:02
2017-06-22 finished!    09:56:04
2017-06-23 finished!    09:56:06
2017-06-24 finished!    09:56:08
2017-06-25 finished!    09:56:09
2017-06-26 finished!    09:56:11
2017-06-27 finished!    09:56:13
2017-06-28 finished!    09:56:14
2017-06-29 finished!    09:56:16
2017-06-30 finished!    09:56:18
2017-07-01 finished!    09:56:20
2017-07-02 finished!    09:56:21
2017-07-03 finished!    09:56:23
2017-07-04 finished!    09:56:26
2017-07-05 finished!    09:56:28
2017-07-06 finished!    09:56:30
2017-07-07 finished!    09:56:32
2017-07-08 finished!    09:56:33
2017-07-09 finished!    09:56:35
2017-07-10 finished!    09:56:37
2017-07-11 finished!    

In [3]:
game_header

GAME_DATE_EST GAME_SEQUENCE     GAME_ID GAME_STATUS_ID  \
0     2013-10-05T00:00:00             1  0011300001              3   
1     2013-10-05T00:00:00             2  0011300002              3   
2     2013-10-05T00:00:00             3  0011300003              3   
3     2013-10-05T00:00:00             4  0011300004              3   
4     2013-10-06T00:00:00             1  0011300005              3   
5     2013-10-06T00:00:00             2  0011300006              3   
6     2013-10-07T00:00:00             3  0011300007              3   
7     2013-10-07T00:00:00             4  0011300008              3   
8     2013-10-07T00:00:00             6  0011300009              3   
9     2013-10-07T00:00:00             7  0011300010              3   
10    2013-10-07T00:00:00             1  0011300011              3   
11    2013-10-07T00:00:00             2  0011300012              3   
12    2013-10-07T00:00:00             5  0011300013              3   
13    2013-10-07T00:00:00             8  0011300014              3   
14    2013-10-08T00:00:00             2  0011300015              3   
15    2013-10-08T00:00:00             3  0011300016              3   
16    2013-10-08T00:00:00             6  0011300017              3   
17    2013-10-08T00:00:00             7  0011300018              3   
18    2013-10-08T00:00:00             4  0011300019              3   
19    2013-10-08T00:00:00             5  0011300020              3   
20    2013-10-08T00:00:00             1  0011300021              3   
21    2013-10-09T00:00:00             1  0011300022              3   
22    2013-10-09T00:00:00             2  0011300023              3   
23    2013-10-09T00:00:00             4  0011300024              3   
24    2013-10-09T00:00:00             6  0011300025              3   
25    2013-10-09T00:00:00             3  0011300026              3   
26    2013-10-09T00:00:00             5  0011300027              3   
27    2013-10-10T00:00:00             3  0011300028              3   
28    2013-10-10T00:00:00             4  0011300029              3   
29    2013-10-10T00:00:00             2  0011300030              3   
...                   ...           ...         ...            ...   
6989  2017-05-12T00:00:00             2  0041600206              3   
6990  2017-05-15T00:00:00             1  0041600207              3   
6991  2017-05-01T00:00:00             1  0041600211              3   
6992  2017-05-03T00:00:00             1  0041600212              3   
6993  2017-05-05T00:00:00             1  0041600213              3   
6994  2017-05-07T00:00:00             1  0041600214              3   
6995  2017-05-02T00:00:00             2  0041600221              3   
6996  2017-05-04T00:00:00             2  0041600222              3   
6997  2017-05-06T00:00:00             1  0041600223              3   
6998  2017-05-08T00:00:00             1  0041600224              3   
6999  2017-05-01T00:00:00             2  0041600231              3   
7000  2017-05-03T00:00:00             2  0041600232              3   
7001  2017-05-05T00:00:00             2  0041600233              3   
7002  2017-05-07T00:00:00             3  0041600234              3   
7003  2017-05-09T00:00:00             2  0041600235              3   
7004  2017-05-11T00:00:00             1  0041600236              3   
7005  2017-05-17T00:00:00             1  0041600301              3   
7006  2017-05-19T00:00:00             1  0041600302              3   
7007  2017-05-21T00:00:00             1  0041600303              3   
7008  2017-05-23T00:00:00             1  0041600304              3   
7009  2017-05-25T00:00:00             1  0041600305              3   
7010  2017-05-14T00:00:00             4  0041600311              3   
7011  2017-05-16T00:00:00             1  0041600312              3   
7012  2017-05-20T00:00:00             1  0041600313              3   
7013  2017-05-22T00:00:00             1  0041600314              3   
7014  2017-06-01T00:00

In [61]:
try:
    # read sql table of game stats logs
    game_stats_logs = pd.read_sql_table('game_stats_logs', conn)
    length_1 = len(game_stats_logs)
    print(str(length_1) + ' player stats loaded.')
except ValueError:
    print('no table yet!')
    length_1 = 0
    # if there is no table yet, create one by getting the first game stats
    initial_game = game_header['GAME_ID'].min()
    game_stats = nba_py.game.Boxscore(initial_game).player_stats()
    # get home team stats
    home_team_id = int(game_header[game_header['GAME_ID'] == initial_game]['HOME_TEAM_ID'])
    home_stats_logs = game_stats[game_stats['TEAM_ID'] == int(home_team_id)].copy()
    # set home team location
    home_stats_logs['LOCATION'] = 'HOME'
    # set home team against team id
    home_stats_logs['AGAINST_TEAM_ID'] = int(game_header[game_header['GAME_ID'] == initial_game]['VISITOR_TEAM_ID'])
    # get away team stats
    away_team_id = int(game_header[game_header['GAME_ID'] == initial_game]['VISITOR_TEAM_ID'])
    away_stats_logs = game_stats[game_stats['TEAM_ID'] == int(away_team_id)].copy()
    # set away team location
    away_stats_logs['LOCATION'] = 'AWAY'
    # set away team against team id
    away_stats_logs['AGAINST_TEAM_ID'] = int(game_header[game_header['GAME_ID'] == initial_game]['HOME_TEAM_ID'])
    # concat home team stats and away team stats
    game_stats_logs = pd.concat([home_stats_logs, away_stats_logs])
    # commit initialized game stats logs to sql table
    game_stats_logs.to_sql('game_stats_logs', conn, index = False, if_exists = 'replace')
    print('game stats logs initialized!')

# ------method 1------for game id in game headers from the max one in sql table
# for i in game_header[game_header['GAME_ID'] >= game_stats_logs['GAME_ID'].max()]['GAME_ID']:

# ------method 2------for game id in game header but not in game stats logs 
for i in game_header['GAME_ID'][game_header['GAME_ID'].isin(game_stats_logs['GAME_ID'].drop_duplicates()) == False]:
    game_stats = nba_py.game.Boxscore(i).player_stats()
    home_team_id = int(game_header[game_header['GAME_ID'] == i]['HOME_TEAM_ID'])
    home_stats_logs = game_stats[game_stats['TEAM_ID'] == int(home_team_id)].copy()
    home_stats_logs['LOCATION'] = 'HOME'
    home_stats_logs['AGAINST_TEAM_ID'] = int(game_header[game_header['GAME_ID'] == i]['VISITOR_TEAM_ID'])
    away_team_id = int(game_header[game_header['GAME_ID'] == i]['VISITOR_TEAM_ID'])
    away_stats_logs = game_stats[game_stats['TEAM_ID'] == int(away_team_id)].copy()
    away_stats_logs['LOCATION'] = 'AWAY'
    away_stats_logs['AGAINST_TEAM_ID'] = int(game_header[game_header['GAME_ID'] == i]['HOME_TEAM_ID'])
    game_stats_logs = game_stats_logs.append(home_stats_logs)
    game_stats_logs = game_stats_logs.append(away_stats_logs)
    print('game ' + i + ' added!    ' + str(datetime.datetime.now().time())[:8])

length_2 = len(game_stats_logs)
# drop duplicate game stats by game id and player id
game_stats_logs = game_stats_logs.drop_duplicates(['GAME_ID', 'PLAYER_ID'])
length_3 = len(game_stats_logs)
print(str(length_2 - length_3) + ' duplicates droped.')
print(str(length_3 - length_1) + ' player stats added.')

# commit new game stats logs to sql table
game_stats_logs.to_sql('game_stats_logs', conn, index = False, if_exists = 'replace')
print(str(length_3) + ' player stats commit complete!')

182419 player stats loaded.
game 0011300114 added!    20:57:51
0 duplicates droped.
0 player stats added.
182419 player stats commit complete!


In [63]:
game_stats_logs

GAME_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_CITY   PLAYER_ID  \
0       0011300001       12321               FBU       Istanbul       42547   
1       0011300001       12321               FBU       Istanbul       42534   
2       0011300001       12321               FBU       Istanbul       42546   
3       0011300001       12321               FBU       Istanbul       42531   
4       0011300001       12321               FBU       Istanbul       42544   
5       0011300001       12321               FBU       Istanbul       42545   
6       0011300001       12321               FBU       Istanbul       42538   
7       0011300001       12321               FBU       Istanbul       42537   
8       0011300001       12321               FBU       Istanbul       42541   
9       0011300001       12321               FBU       Istanbul       42542   
10      0011300001       12321               FBU       Istanbul       42536   
11      0011300001       12321               FBU       Istanbul       42540   
12      0011300001       12321               FBU       Istanbul       42535   
13      0011300001       12321               FBU       Istanbul  1962936724   
14      0011300001  1610612760               OKC  Oklahoma City      203460   
15      0011300001  1610612760               OKC  Oklahoma City         965   
16      0011300001  1610612760               OKC  Oklahoma City      203197   
17      0011300001  1610612760               OKC  Oklahoma City      201934   
18      0011300001  1610612760               OKC  Oklahoma City      203087   
19      0011300001  1610612760               OKC  Oklahoma City        2570   
20      0011300001  1610612760               OKC  Oklahoma City      201142   
21      0011300001  1610612760               OKC  Oklahoma City        2555   
22      0011300001  1610612760               OKC  Oklahoma City      203103   
23      0011300001  1610612760               OKC  Oklahoma City      202704   
24      0011300001  1610612760               OKC  Oklahoma City      201586   
25      0011300001  1610612760               OKC  Oklahoma City      203500   
26      0011300001  1610612760               OKC  Oklahoma City      200757   
27      0011300002  1610612754               IND        Indiana      201228   
28      0011300002  1610612754               IND        Indiana      203142   
29      0011300002  1610612754               IND        Indiana      101122   
...            ...         ...               ...            ...         ...   
182389  0041600404  1610612744               GSW   Golden State      203949   
182390  0041600404  1610612744               GSW   Golden State        2440   
182391  0041600404  1610612744               GSW   Golden State      203546   
182392  0041600404  1610612744               GSW   Golden State        2561   
182393  0041600405  1610612744               GSW   Golden State      201142   
182394  0041600405  1610612744               GSW   Golden State      203110   
182395  0041600405  1610612744               GSW   Golden State        2585   
182396  0041600405  1610612744               GSW   Golden State      202691   
182397  0041600405  1610612744               GSW   Golden State      201939   
182398  0041600405  1610612744               GSW   Golden State        2738   
182399  0041600405  1610612744               GSW   Golden State        2733   
182400  0041600405  1610612744               GSW   Golden State     1627775   
182401  0041600405  1610612744               GSW   Golden State        2440   
182402  0041600405  1610612744               GSW   Golden State        2561   
182403  0041600405  1610612744               GSW   Golden State      203546   
182404  0041600405  1610612744               GSW   Golden State      203949   
182405  0041600405  1610612744               GSW   Golden State      201580   
182406  0041600405  1610612739               CLE      Cleveland        2544   
182407  0041600405  1610612739               CLE      Clevela